In [50]:
import pandas as pd
import numpy as np
import math

In [51]:
# definir arquivo Pickle de entrada
#arquivo = 'dados/afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'dados/cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

In [52]:
df = pd.read_pickle(arquivo)
df.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,15925,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,11497,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,7515,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,7370,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [53]:
%%time
df2 = df.head(50000).copy()
#df2 = df.copy()
menor = df2['QTDE'].min()
maior = df2['QTDE'].max()
print(df2.shape[0], menor, maior, math.ceil(np.sqrt(maior / menor)))
#df2['TAM'] = df2['QTDE'].apply(lambda x: math.ceil(np.sqrt(x / menor)))
df2['TEMP'] = df2[['SEXO', 'QTDE']].apply(
    lambda x: [x[0]] * math.ceil(np.sqrt(x[1] / menor)), axis=1)
dfe = df2.explode('TEMP').drop(['TEMP'], axis=1) #.reset_index()
dfe.head(1000)

# usar os dados expandidos invés dos agregados
dfo = df # guardar o dataframe original
#df = dfe # apontar para o dataframe expandido
df = dfe.sample(frac=1) # embaralhar as linhas

41146 1 20389 143
CPU times: user 387 ms, sys: 3.95 ms, total: 391 ms
Wall time: 387 ms


In [54]:
X = df.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (82709, 16) (82709,)


In [55]:
# treinar com todos os dados disponíveis

X_train, y_train = X, y

print("Shapes:", X_train.shape, y_train.shape)

Shapes: (82709, 16) (82709,)


https://automl.github.io/auto-sklearn/master/manual.html

In [56]:
# https://automl.github.io/auto-sklearn/master/installation.html

#!curl -k https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
#!pip3 install auto-sklearn

In [57]:
import autosklearn.classification

In [58]:
X_train.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,
DARLON,14,15,12,18,1,4,0,0,0,0,0,0,0,0,0,0
ANDRELISE,5,19,9,12,5,18,4,14,1,0,0,0,0,0,0,0
ALYSSON,14,15,19,19,25,12,1,0,0,0,0,0,0,0,0,0
PETER,18,5,20,5,16,0,0,0,0,0,0,0,0,0,0,0
JACKELYNE,5,14,25,12,5,11,3,1,10,0,0,0,0,0,0,0


In [59]:
y_train[:5]

NOME
DARLON       1
ANDRELISE    0
ALYSSON      1
PETER        1
JACKELYNE    0
Name: SEXO, dtype: uint8

In [60]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60 * 60, # 1 hora
    per_run_time_limit=5 * 60, # 5 min
    n_jobs=-1,
    memory_limit=2048, # 2 GB RAM
    seed=42,
    include_preprocessors=["no_preprocessing"]
)
automl

AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=2048, n_jobs=-1, per_run_time_limit=300,
                      seed=42)

In [61]:
%%time
automl.fit(X_train, y_train)

CPU times: user 20min 16s, sys: 38.4 s, total: 20min 54s
Wall time: 1h 2s


AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=2048, n_jobs=-1, per_run_time_limit=300,
                      seed=42)

In [62]:
print(automl.show_models())

[(0.140000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'median', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:random_forest:bootstrap': 'False', 'classifier:random_forest:criterion': 'entropy', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.754262775877861, 'classifier:random_forest:max_leaf_nodes': 'None', 'classifier:random_forest:min_impurity_decrease': 0.0, 'classifier:random_forest:min_samples_leaf': 1, 'classifier:random_forest:min_samples_split': 3, 'classifier:random_forest:min_weight_fraction_leaf': 0.0, 'data_p

In [63]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: c86a5f0c-c573-11eb-bde3-f52637a0d4db
  Metric: accuracy
  Best validation score: 0.999304
  Number of target algorithm runs: 342
  Number of successful target algorithm runs: 247
  Number of crashed target algorithm runs: 45
  Number of target algorithms that exceeded the time limit: 50
  Number of target algorithms that exceeded the memory limit: 0



In [64]:
losses_and_configurations = [
    (run_value.cost, run_key.config_id)
    for run_key, run_value in automl.automl_.runhistory_.data.items()
]
losses_and_configurations.sort()
print("Lowest loss:", losses_and_configurations[0][0])
print("Best configuration:", automl.automl_.runhistory_.ids_config[losses_and_configurations[0][1]])

Lowest loss: 0.0006961236901883439
Best configuration: Configuration:
  balancing:strategy, Value: 'none'
  classifier:__choice__, Value: 'random_forest'
  classifier:random_forest:bootstrap, Value: 'False'
  classifier:random_forest:criterion, Value: 'entropy'
  classifier:random_forest:max_depth, Constant: 'None'
  classifier:random_forest:max_features, Value: 0.7588185975561116
  classifier:random_forest:max_leaf_nodes, Constant: 'None'
  classifier:random_forest:min_impurity_decrease, Constant: 0.0
  classifier:random_forest:min_samples_leaf, Value: 1
  classifier:random_forest:min_samples_split, Value: 4
  classifier:random_forest:min_weight_fraction_leaf, Constant: 0.0
  data_preprocessing:categorical_transformer:categorical_encoding:__choice__, Value: 'no_encoding'
  data_preprocessing:categorical_transformer:category_coalescence:__choice__, Value: 'minority_coalescer'
  data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction, Value: 0

In [65]:
automl

AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=2048, n_jobs=-1, per_run_time_limit=300,
                      seed=42)

In [66]:
# gravar em disco o modelo treinado
from joblib import dump
dump(automl, 'modelos/modelo-genero-nome-autoskl2.joblib.bz2', compress=('bz2', 9))

['modelos/modelo-genero-nome-autoskl2.joblib.bz2']

In [22]:
# carregar dados de teste: definir arquivo Pickle de entrada
arquivo = 'dados/afastamentos-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'dados/cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

In [23]:
df2 = pd.read_pickle(arquivo)
df2.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,2937,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,2766,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1,1257,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,1,1014,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,1,993,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
df2.shape

(8711, 18)

In [25]:
X_test = df2.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y_test = df2['SEXO'] # apenas a coluna alvo

In [26]:
y_pred = automl.predict(X_test)
y_pred

array([0, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [27]:
from sklearn.metrics import accuracy_score

print("Accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Accuracy score: 99.94 %


In [28]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Confusion matrix:
 [[4382    2]
 [   3 4324]]
